In [1]:
import pubchempy as pcp
import requests
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',200,'display.max_columns',50)
import csv
import time
import pickle
from tqdm import tqdm
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import QED
from rdkit.Chem.Descriptors import MolWt
from rdkit.Chem import RDConfig
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir,'SA_Score'))
from sascorer import calculateScore

In [22]:
data = pd.read_csv("BindingDB_curation_20200608.csv")

In [23]:
data.head()

,Ligand SMILES,BindingDB Target Chain Sequence,IC50 (nM),Ligand_len,Target_len
0,O=C1CCCC2=C1C1(CCS(=O)(=O)C1)N=C(Nc1nc3ccccc3o...,MAALRQPQVAELLAEARRAFREEFGAEPELAVSAPGRVNLIGEHTD...,6676.9,50.0,392.0
1,CC(C)[C@H](NC(C)=O)C(=O)N[C@@H](Cc1ccccc1)[C@@...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,8.5,90.0,99.0
2,CCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1)[...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,177.0,95.0,99.0
3,COCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,164.0,99.0,99.0
4,CCOC(=O)N[C@@H](C(C)C)C(=O)NN(C[C@H](O)[C@H](C...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,56.0,92.0,99.0


In [24]:
data.shape

(712717, 5)

In [29]:
data["Ligand SMILES"].unique().shape

(406556,)

In [32]:
data.columns

Index(['Ligand SMILES', 'BindingDB Target Chain  Sequence', 'IC50 (nM)',
       'Ligand_len', 'Target_len'],
      dtype='object')

In [34]:
data["BindingDB Target Chain  Sequence"].unique().shape

(4049,)

In [40]:
over100 = data[data["IC50 (nM)"]>100]

In [41]:
over100["BindingDB Target Chain  Sequence"].unique().shape

(3716,)

In [42]:
over100["Ligand SMILES"].unique().shape

(233066,)

In [44]:
over100

,Ligand SMILES,BindingDB Target Chain Sequence,IC50 (nM),Ligand_len,Target_len
0,O=C1CCCC2=C1C1(CCS(=O)(=O)C1)N=C(Nc1nc3ccccc3o...,MAALRQPQVAELLAEARRAFREEFGAEPELAVSAPGRVNLIGEHTD...,6676.9,50.0,392.0
2,CCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1)[...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,177.0,95.0,99.0
3,COCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,164.0,99.0,99.0
7,CCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1)[...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,150.0,97.0,99.0
9,CCOC(=O)N[C@@H](C(C)C)C(=O)NN(C[C@H](O)[C@H](C...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,134.0,95.0,99.0
...,...,...,...,...,...
712712,CCN(CCCOc1ccc(cc1)-c1coc2cc(OCCCN(CC)Cc3ccccc3...,MKILDALLFPVIFIMFFIHLSIAQTDPELTIMTRLGQVQGTRLPVP...,160.0,72.0,633.0
712713,CCN(CCCCCCOc1cc(O)c2c(c1)occ(-c1ccc(O)cc1)c2=O...,MKILDALLFPVIFIMFFIHLSIAQTDPELTIMTRLGQVQGTRLPVP...,4980.0,56.0,633.0
712714,COc1ccc2c(c1)[nH]c1cccc(CN(C(C)C)C(=O)CCO[N+](...,EEDIIITTKNGKVRGMNLPVLGGTVTAFLGIPYAQPPLGRLRFKKP...,2500.0,57.0,574.0
712715,COc1ccc2c(c1)oc1cccc(CN(C(C)C)C(=O)CCO[N+]([O-...,MKILDALLFPVIFIMFFIHLSIAQTDPELTIMTRLGQVQGTRLPVP...,180.0,54.0,633.0


In [46]:
data.head()

,Ligand SMILES,BindingDB Target Chain Sequence,IC50 (nM),Ligand_len,Target_len
0,O=C1CCCC2=C1C1(CCS(=O)(=O)C1)N=C(Nc1nc3ccccc3o...,MAALRQPQVAELLAEARRAFREEFGAEPELAVSAPGRVNLIGEHTD...,6676.9,50.0,392.0
1,CC(C)[C@H](NC(C)=O)C(=O)N[C@@H](Cc1ccccc1)[C@@...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,8.5,90.0,99.0
2,CCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1)[...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,177.0,95.0,99.0
3,COCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,164.0,99.0,99.0
4,CCOC(=O)N[C@@H](C(C)C)C(=O)NN(C[C@H](O)[C@H](C...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,56.0,92.0,99.0


In [59]:
data.iloc[0]["BindingDB Target Chain  Sequence"]

'MAALRQPQVAELLAEARRAFREEFGAEPELAVSAPGRVNLIGEHTDYNQGLVLPMALELMTVLVGSPRKDGLVSLLTTSEGADEPQRLQFPLPTAQRSLEPGTPRWANYVKGVIQYYPAAPLPGFSAVVVSSVPLGGGLSSSASLEVATYTFLQQLCPDSGTIAARAQVCQQAEHSFAGMPCGIMDQFISLMGQKGHALLIDCRSLETSLVPLSDPKLAVLITNSNVRHSLASSEYPVRRRQCEEVARALGKESLREVQLEELEAARDLVSKEGFRRARHVVGEIRRTAQAAAALRRGDYRAFGRLMVESHRSLRDDYEVSCPELDQLVEAALAVPGVYGSRMTGGGFGGCTVTLLEASAAPHAMRHIQEHYGGTATFYLSQAADGAKVLCL'

In [47]:
query = data.iloc[0]["Ligand SMILES"]

In [50]:
query = data.iloc[0]["Ligand SMILES"]
url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastsimilarity_2d/smiles/{}/cids/JSON?Threshold=95&MaxRecords=10".format(query)

In [52]:
res = requests.get(url)

In [54]:
d = json.loads(res.text)

In [56]:
d["IdentifierList"]

{'CID': [44623880, 44640149, 44640151]}

In [57]:
def get_cids(query, by="SMILES"):
    if by == "SMILES":
        url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastsimilarity_2d/smiles/{}/cids/JSON?Threshold=95&MaxRecords=10".format(query)
        res = requests.get(url)
        d = json.loads(res.text)
        return d["IdentifierList"]
    else:
        return None

In [58]:
get_cids(query)

{'CID': [44623880, 44640149, 44640151]}

In [61]:
cids = []
for i in tqdm(range(500)):
    query = data.iloc[i]["Ligand SMILES"]
    cids.append(get_cids(query))

 20%|█▉        | 99/500 [04:27<18:04,  2.71s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home2/glee/.conda/envs/drug2/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-61-410c0ef3ebe3>", line 4, in <module>
    cids.append(get_cids(query))
  File "<ipython-input-57-ee52f7239d67>", line 4, in get_cids
    res = requests.get(url)
  File "/home2/glee/.conda/envs/drug2/lib/python3.6/site-packages/requests/api.py", line 76, in get
    return request('get', url, params=params, **kwargs)
  File "/home2/glee/.conda/envs/drug2/lib/python3.6/site-packages/requests/api.py", line 61, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home2/glee/.conda/envs/drug2/lib/python3.6/site-packages/requests/sessions.py", line 530, in request
    resp = self.send(prep, **send_kwargs)
  File "/home2/glee/.conda/envs/drug2/lib/python3.6/site-packages/requests/sessions.py", line 643, in send
    r = adapter.send(r

KeyboardInterrupt: 

In [62]:
cids[:10]

[{'CID': [44623880, 44640149, 44640151]},
 {'CID': [65023,
   461984,
   461965,
   461972,
   461966,
   461982,
   480069,
   44328419,
   44338669,
   44338692]},
 {'CID': [65023,
   461984,
   461965,
   461990,
   461969,
   461972,
   461976,
   461979,
   461983,
   461988]},
 {'CID': [461984,
   461965,
   461990,
   461969,
   461972,
   461976,
   461979,
   461983,
   461988,
   461982]},
 {'CID': [65023,
   461984,
   461965,
   461990,
   461969,
   461972,
   461976,
   461979,
   461983,
   461988]},
 {'CID': [461984,
   461965,
   461990,
   461969,
   461972,
   461976,
   461979,
   461983,
   461988,
   461982]},
 {'CID': [65023,
   461984,
   461965,
   461990,
   461969,
   461972,
   461976,
   461979,
   461983,
   461988]},
 {'CID': [461984,
   461965,
   461990,
   461969,
   461972,
   461976,
   461979,
   461983,
   461988,
   461982]},
 {'CID': [461984,
   461965,
   461990,
   461969,
   461972,
   461976,
   461979,
   461983,
   461988,
   461982]},
 {'C

In [7]:
alldata = pd.read_csv("data/BindingDB_All_202404.tsv", sep="\t", error_bad_lines=False, nrows=50000)

/home2/glee/.conda/envs/drug2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10,11,21,45,46,47,48,57,58,59,62,63,64,65,66,67,69,70,71,72) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
alldata.columns

Index(['BindingDB Reactant_set_id', 'Ligand SMILES', 'Ligand InChI',
       'Ligand InChI Key', 'BindingDB MonomerID', 'BindingDB Ligand Name',
       'Target Name',
       'Target Source Organism According to Curator or DataSource', 'Ki (nM)',
       'IC50 (nM)',
       ...
       'UniProt (SwissProt) Recommended Name of Target Chain.12',
       'UniProt (SwissProt) Entry Name of Target Chain.12',
       'UniProt (SwissProt) Primary ID of Target Chain.12',
       'UniProt (SwissProt) Secondary ID(s) of Target Chain.12',
       'UniProt (SwissProt) Alternative ID(s) of Target Chain.12',
       'UniProt (TrEMBL) Submitted Name of Target Chain.12',
       'UniProt (TrEMBL) Entry Name of Target Chain.12',
       'UniProt (TrEMBL) Primary ID of Target Chain.12',
       'UniProt (TrEMBL) Secondary ID(s) of Target Chain.12',
       'UniProt (TrEMBL) Alternative ID(s) of Target Chain.12'],
      dtype='object', length=194)

In [42]:
def get_activity(cid):
    query = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/'+str(cid)+'/assaysummary/JSON'
    res = requests.get(query)
    d = json.loads(res.text)
    columns = np.array(d['Table']['Columns']['Column'])
    results = []
    target_dict = {}
    for i in range(len(d['Table']['Row'])):
        temp = list(d['Table']['Row'][i].values())[0]
        if 'Active' in temp and temp[8]!='' and temp[9]!='':
            results.append(temp)
            target_dict[temp[6]] = {}
    for i in results:
        #print(i[8],i[10],i[9])
        if i[8] in target_dict[i[6]].keys():
            target_dict[i[6]][i[8]].append(float(i[7]))
        else:
            target_dict[i[6]][i[8]] = []
            target_dict[i[6]][i[8]].append(float(i[7]))
    for i in results:
        target_dict[i[6]][i[8]] = np.mean(target_dict[i[6]][i[8]])
    ret = target_dict
    return ret

In [12]:
drugs = ['Allopurinol', 'Doxycycline', 'Lidocaine', 'Mazindol', 'Topiramate', 'Valproic acid', 'Zidovudine']
drug_cids = [135401907, 54671203, 3676, 4020, 5284627, 3121, 35370]

In [13]:
query = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/'+str(drug_cids[0])+'/assaysummary/JSON'
res = requests.get(query)
d = json.loads(res.text)
columns = np.array(d['Table']['Columns']['Column'])

In [14]:
columns

array(['AID', 'Panel Member ID', 'SID', 'CID', 'Activity Outcome',
       'Target GI', 'Target GeneID', 'Activity Value [uM]',
       'Activity Name', 'Assay Name', 'Assay Type', 'PubMed ID', 'RNAi'],
      dtype='<U19')

In [38]:
aaa = pd.DataFrame([cell["Cell"] for cell in d['Table']['Row']], columns = d['Table']['Columns']['Column']).set_index("AID")

In [42]:
aaa[aaa["Activity Outcome"]=="Active"]

,Panel Member ID,SID,CID,Activity Outcome,Target GI,Target GeneID,Activity Value [uM],Activity Name,Assay Name,Assay Type,PubMed ID,RNAi
AID,,,,,,,,,,,,
374,,8139898,135401907,Active,4758204,1843,,,In vitro Primary HTS Assay for MKP-1,Screening,,
1195,,48415533,135401907,Active,,,,,DSSTox (FDAMDD) FDA Maximum (Recommended) Dail...,Other,,
1822,,8139898,135401907,Active,23505220,813594,,,QFRET-based primary biochemical high throughpu...,Screening,,
1996,,56424020,135401907,Active,,,,,Aqueous Solubility from MLSMR Stock Solutions,Other,,
219599,,103533730,135401907,Active,109940048,280960,0.26,IC50,In vitro inhibitory concentration against Xant...,Confirmatory,11294382,
219739,,103533730,135401907,Active,2506326,7498,4.2,IC50,Inhibitory activity against Xanthine Oxidase,Confirmatory,8691450,
219741,,103533730,135401907,Active,2506326,7498,7,Ki,Inhibitory activity against Xanthine Oxidase,Confirmatory,8691450,
287937,,103533730,135401907,Active,2506326,7498,7.82,IC50,Inhibition of human xanthine oxidase,Confirmatory,17379526,
295041,,103533730,135401907,Active,2506326,7498,2.2,IC50,Inhibition of Xanthine oxidase,Confirmatory,17346963,


In [45]:
target_dict = {}
results = []
for i in range(len(d['Table']['Row'])):
    temp = list(d['Table']['Row'][i].values())[0]
    if 'Active' in temp and temp[8]!='' and temp[9]!='':
        results.append(temp)
        target_dict[temp[8]] = {}

In [49]:
results[0]

['219595',
 '',
 '103533730',
 '135401907',
 'Active',
 '2506326',
 '7498',
 '34.91',
 'IC50',
 'Inhibition of Xanthine oxidase',
 'Confirmatory',
 '12873513',
 '']

In [50]:
target_dict

{'IC50': {}, 'Ki': {}, 'Km': {}, 'Activity': {}, 'Potency': {}, 'Kd': {}}

In [51]:
activities = get_activity(drug_cids[2])

In [52]:
activities

{'': {'EC50': 0.017, 'Potency': 10.294966666666667},
 '6335': {'IC50': 29.0},
 '6329': {'IC50': 2.0},
 '6328': {'IC50': 10.5}}